In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier

In [2]:
!pip install lightgbm

  Using cached lightgbm-3.1.1.tar.gz (588 kB)
  Running setup.py clean for lightgbm
Failed to build lightgbm
    Running setup.py install for lightgbm: started
    Running setup.py install for lightgbm: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'c:\users\dell\appdata\local\programs\python\python38-32\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\DELL\\AppData\\Local\\Temp\\pip-install-_sto5mas\\lightgbm\\setup.py'"'"'; __file__='"'"'C:\\Users\\DELL\\AppData\\Local\\Temp\\pip-install-_sto5mas\\lightgbm\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\DELL\AppData\Local\Temp\pip-wheel-42jsw407'
       cwd: C:\Users\DELL\AppData\Local\Temp\pip-install-_sto5mas\lightgbm\
  Complete output (53 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib
  creating build\lib\lightgbm
  copying lightgbm\basic.py -> build\lib\lightgbm
  copying lightgbm\callback.py -> build\lib\lightgbm
  copying lightgbm\compat.py -> build\lib\lightgbm
  copying 

In [5]:
# Load in the training set
training_set = pd.read_csv("train.csv")
training_set.hist_user_behavior_reason_start = training_set.hist_user_behavior_reason_start.astype('category')
training_set.hist_user_behavior_reason_end = training_set.hist_user_behavior_reason_end.astype('category')
training_set.context_type = training_set.context_type.astype('category')
training_set.date = training_set.date.apply(pd.to_datetime)

# Load in the track features
track_data = pd.read_csv("track.csv")

# Using skip_2 as the ground truth
training_set['skipped'] = (training_set.skip_2 | training_set.skip_1).astype('int32')
training_set = training_set.drop(columns=['skip_1','skip_2','skip_3','not_skipped'])
training_set.head()

c:\users\dell\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3,4,5,6,7,14,16,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,session_id,session_position,session_length,track_id_clean,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end,skipped
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1.0,20.0,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,0.0,0.0,0.0,0.0,0.0,0.0,True,16.0,2018-07-15,True,editorial_playlist,trackdone,trackdone,0
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2.0,20.0,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,0.0,1.0,0.0,0.0,0.0,0.0,True,16.0,2018-07-15,True,editorial_playlist,trackdone,trackdone,0
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3.0,20.0,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,0.0,1.0,0.0,0.0,0.0,0.0,True,16.0,2018-07-15,True,editorial_playlist,trackdone,trackdone,0
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4.0,20.0,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,0.0,1.0,0.0,0.0,0.0,0.0,True,16.0,2018-07-15,True,editorial_playlist,trackdone,trackdone,0
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5.0,20.0,t_64f3743c-f624-46bb-a579-0f3f9a07a123,0.0,1.0,0.0,0.0,0.0,0.0,True,16.0,2018-07-15,True,editorial_playlist,trackdone,trackdone,0


In [7]:
training_set=training_set.dropna()

In [8]:
training_set.isnull().sum()

session_id                         0
session_position                   0
session_length                     0
track_id_clean                     0
context_switch                     0
no_pause_before_play               0
short_pause_before_play            0
long_pause_before_play             0
hist_user_behavior_n_seekfwd       0
hist_user_behavior_n_seekback      0
hist_user_behavior_is_shuffle      0
hour_of_day                        0
date                               0
premium                            0
context_type                       0
hist_user_behavior_reason_start    0
hist_user_behavior_reason_end      0
skipped                            0
dtype: int64

In [9]:
track_data.isnull().sum()

track_id                  0
duration                  0
release_year              0
us_popularity_estimate    0
acousticness              0
beat_strength             0
bounciness                0
danceability              0
dyn_range_mean            0
energy                    0
flatness                  0
instrumentalness          0
key                       0
liveness                  0
loudness                  0
mechanism                 0
is_major                  0
organism                  0
speechiness               0
tempo                     0
time_signature            0
valence                   0
acoustic_vector_0         0
acoustic_vector_1         0
acoustic_vector_2         0
acoustic_vector_3         0
acoustic_vector_4         0
acoustic_vector_5         0
acoustic_vector_6         0
acoustic_vector_7         0
dtype: int64

In [9]:
track_data.head()

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_7f435320-5d3b-497d-9346-0c97429af2b9,3.866097,0.441176,-0.403765,0.075846,0.499848,0.494943,0.461215,-0.208611,0.563252,...,0.571429,0.468440,1.034540,-1.040880,-1.281460,-0.375163,1.442307,0.045940,-0.443245,-0.243705
1,t_2a7b6393-ebec-4e1f-a53f-2ad842532c99,1.512969,0.308824,0.314153,0.085482,0.450139,0.457034,0.663219,-0.305010,0.767307,...,0.571429,0.655369,0.834594,-0.653568,-0.483168,-0.538935,1.247153,0.414248,-0.468285,-0.931875
2,t_949441c4-4920-4eed-b929-2d67d37b7bd9,-1.332280,0.485294,0.001032,0.002274,0.243778,0.229295,0.290262,-1.181899,0.958647,...,0.571429,0.293100,0.846650,0.123691,-2.692681,-0.770798,1.533066,-0.173705,-0.402540,1.181761
3,t_63eae5ac-744a-46ee-b719-0db8e70d6f7f,-0.712162,0.485294,0.558944,0.005024,0.338580,0.331703,0.348846,-0.795663,0.886674,...,0.571429,0.387131,0.920349,0.335177,-2.631787,-1.033223,1.549514,-0.036511,-0.488062,1.028883
4,t_a0747d21-6042-4af7-a21d-aa7cd1a8094e,-1.629474,0.573529,-0.356946,0.962717,0.130125,0.092335,0.170027,-2.024206,0.005674,...,0.428571,0.030858,2.238796,-3.259609,-2.378984,1.288895,-1.053248,-3.132837,0.946820,-0.623612


In [10]:
track_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316702 entries, 0 to 316701
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   track_id                316702 non-null  object 
 1   duration                316702 non-null  float64
 2   release_year            316702 non-null  float64
 3   us_popularity_estimate  316702 non-null  float64
 4   acousticness            316702 non-null  float64
 5   beat_strength           316702 non-null  float64
 6   bounciness              316702 non-null  float64
 7   danceability            316702 non-null  float64
 8   dyn_range_mean          316702 non-null  float64
 9   energy                  316702 non-null  float64
 10  flatness                316702 non-null  float64
 11  instrumentalness        316702 non-null  float64
 12  key                     316702 non-null  float64
 13  liveness                316702 non-null  float64
 14  loudness            

In [8]:
# Convert the columns to all be consistantly numeric
track_data.iloc[:,1:] = track_data.iloc[:,1:].astype('float32')
track_data.head()

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_7f435320-5d3b-497d-9346-0c97429af2b9,3.866097,0.441176,-0.403765,0.075846,0.499848,0.494943,0.461215,-0.208611,0.563252,...,0.571429,0.468440,1.034540,-1.040880,-1.281460,-0.375163,1.442307,0.045940,-0.443245,-0.243705
1,t_2a7b6393-ebec-4e1f-a53f-2ad842532c99,1.512969,0.308824,0.314153,0.085482,0.450139,0.457034,0.663219,-0.305010,0.767307,...,0.571429,0.655369,0.834594,-0.653568,-0.483167,-0.538935,1.247153,0.414248,-0.468285,-0.931875
2,t_949441c4-4920-4eed-b929-2d67d37b7bd9,-1.332280,0.485294,0.001032,0.002274,0.243778,0.229295,0.290262,-1.181899,0.958647,...,0.571429,0.293100,0.846650,0.123691,-2.692681,-0.770798,1.533066,-0.173705,-0.402540,1.181761
3,t_63eae5ac-744a-46ee-b719-0db8e70d6f7f,-0.712162,0.485294,0.558944,0.005024,0.338580,0.331703,0.348846,-0.795663,0.886674,...,0.571429,0.387131,0.920349,0.335177,-2.631788,-1.033223,1.549514,-0.036511,-0.488062,1.028883
4,t_a0747d21-6042-4af7-a21d-aa7cd1a8094e,-1.629474,0.573529,-0.356946,0.962717,0.130125,0.092335,0.170027,-2.024206,0.005674,...,0.428571,0.030858,2.238796,-3.259609,-2.378984,1.288895,-1.053248,-3.132838,0.946820,-0.623612


In [11]:
# Join the two together
session_data = (
    training_set[['session_id','session_position','track_id_clean', 'skipped']]
    .merge(
        track_data,
        how='left',
        left_on=['track_id_clean'],
        right_on=['track_id']
    ).drop(columns='track_id_clean')
)
session_data.head()

,session_id,session_position,skipped,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1.0,0,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,-0.527006,1.0,0.693523,0.015848,0.438551,0.473455,...,0.571429,0.152255,-1.332878,0.813641,0.440978,0.131989,-0.844142,-0.143796,-0.599983,0.566763
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2.0,0,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,0.165861,1.0,0.644089,0.061811,0.654804,0.735661,...,0.571429,0.337152,-1.090160,0.735187,0.773410,-0.065046,-0.971432,0.160804,-0.631676,0.428458
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3.0,0,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,0.098324,1.0,0.715568,0.354116,0.532155,0.540411,...,0.571429,0.373862,-1.158831,0.776265,0.534293,-0.084075,-0.758170,0.142758,-0.959950,0.496833
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4.0,0,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,-0.652071,1.0,0.712150,0.769225,0.641756,0.729224,...,0.571429,0.649420,-1.069886,0.575604,0.685594,-0.055785,-0.997492,0.318548,-0.629268,0.797080
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5.0,0,t_64f3743c-f624-46bb-a579-0f3f9a07a123,-0.154756,1.0,0.714545,0.006602,0.732428,0.794881,...,0.571429,0.652921,-1.458160,0.623033,0.359345,0.291053,-0.843922,-0.390072,-1.023939,0.498887


In [13]:
full_session_data = pd.concat((
    session_data,
    session_data.drop(columns=['track_id'])\
        .groupby('session_id')\
        .shift(1)\
        .rename(columns={c:f"previous_{c}" for c in session_data.columns})
), axis=1)
full_session_data.head()

,session_id,session_position,skipped,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,...,previous_time_signature,previous_valence,previous_acoustic_vector_0,previous_acoustic_vector_1,previous_acoustic_vector_2,previous_acoustic_vector_3,previous_acoustic_vector_4,previous_acoustic_vector_5,previous_acoustic_vector_6,previous_acoustic_vector_7
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1.0,0,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,-0.527006,1.0,0.693523,0.015848,0.438551,0.473455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2.0,0,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,0.165861,1.0,0.644089,0.061811,0.654804,0.735661,...,0.571429,0.152255,-1.332878,0.813641,0.440978,0.131989,-0.844142,-0.143796,-0.599983,0.566763
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3.0,0,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,0.098324,1.0,0.715568,0.354116,0.532155,0.540411,...,0.571429,0.337152,-1.090160,0.735187,0.773410,-0.065046,-0.971432,0.160804,-0.631676,0.428458
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4.0,0,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,-0.652071,1.0,0.712150,0.769225,0.641756,0.729224,...,0.571429,0.373862,-1.158831,0.776265,0.534293,-0.084075,-0.758170,0.142758,-0.959950,0.496833
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5.0,0,t_64f3743c-f624-46bb-a579-0f3f9a07a123,-0.154756,1.0,0.714545,0.006602,0.732428,0.794881,...,0.571429,0.649420,-1.069886,0.575604,0.685594,-0.055785,-0.997492,0.318548,-0.629268,0.797080


In [14]:
# Examine the columns
full_session_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167880 entries, 0 to 167879
Data columns (total 64 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   session_id                       167880 non-null  object 
 1   session_position                 167880 non-null  float64
 2   skipped                          167880 non-null  int32  
 3   track_id                         154360 non-null  object 
 4   duration                         154360 non-null  float64
 5   release_year                     154360 non-null  float64
 6   us_popularity_estimate           154360 non-null  float64
 7   acousticness                     154360 non-null  float64
 8   beat_strength                    154360 non-null  float64
 9   bounciness                       154360 non-null  float64
 10  danceability                     154360 non-null  float64
 11  dyn_range_mean                   154360 non-null  float64
 12  en

In [15]:
# Split the data:
# df -> X, y -> Train, Test, Val
X = full_session_data.drop(
    columns=["skipped","session_id","track_id"]
).fillna(-9999)
y = full_session_data.skipped
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=2
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=2
)
X.head(20)

,session_position,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,previous_time_signature,previous_valence,previous_acoustic_vector_0,previous_acoustic_vector_1,previous_acoustic_vector_2,previous_acoustic_vector_3,previous_acoustic_vector_4,previous_acoustic_vector_5,previous_acoustic_vector_6,previous_acoustic_vector_7
0,1.0,-0.527006,1.0,0.693523,0.015848,0.438551,0.473455,0.653119,-0.148759,0.553465,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
1,2.0,0.165861,1.0,0.644089,0.061811,0.654804,0.735661,0.877393,1.327177,0.726828,...,0.571429,0.152255,-1.332878,0.813641,0.440978,0.131989,-0.844142,-0.143796,-0.599983,0.566763
2,3.0,0.098324,1.0,0.715568,0.354116,0.532155,0.540411,0.679719,0.015381,0.563009,...,0.571429,0.337152,-1.090160,0.735187,0.773410,-0.065046,-0.971432,0.160804,-0.631676,0.428458
3,4.0,-0.652071,1.0,0.712150,0.769225,0.641756,0.729224,0.864881,1.318616,0.529484,...,0.571429,0.373862,-1.158831,0.776265,0.534293,-0.084075,-0.758170,0.142758,-0.959950,0.496833
4,5.0,-0.154756,1.0,0.714545,0.006602,0.732428,0.794881,0.857778,1.680245,0.650057,...,0.571429,0.649420,-1.069886,0.575604,0.685594,-0.055785,-0.997492,0.318548,-0.629268,0.797080
5,6.0,-0.079714,1.0,0.713925,0.098337,0.548632,0.658323,0.596480,0.966162,0.557129,...,0.571429,0.652921,-1.458160,0.623033,0.359345,0.291053,-0.843922,-0.390072,-1.023939,0.498887
6,7.0,-0.449002,1.0,0.619747,0.449629,0.487708,0.608612,0.557520,0.730798,0.469350,...,0.714286,0.661081,-1.336988,0.457133,1.093895,0.837688,-0.708546,0.469779,0.643404,0.871075
7,8.0,-9999.000000,-9999.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,...,0.571429,0.109420,-1.172783,0.590866,0.471368,0.063138,-0.797903,-0.042197,-0.900465,0.552355
8,9.0,-0.060523,1.0,0.714512,0.030127,0.818798,0.879947,0.964383,2.835511,0.555280,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
9,10.0,-1.993436,1.0,0.686116,0.332467,0.751500,0.843354,0.914082,2.600974,0.239318,...,0.571429,0.338321,-1.163279,0.544965,0.440833,0.239874,-0.611950,-0.084447,-0.734181,0.432584


In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
# Train a Logistic Regression model and score
scaler = StandardScaler().fit(X_train)
sX_train = scaler.transform(X_train)
sX_val = scaler.transform(X_val)
sX_test = scaler.transform(X_test)

log = LogisticRegressionCV(
    cv=3
).fit(
    sX_train,
    y_train
)

print("Log Train score: %s" % log.score(sX_train,y_train))
print("Log Val score:   %s" % log.score(sX_val,y_val))
print("Log Test score:  %s" % log.score(sX_test,y_test))

Log Train score: 0.5337900095864784
Log Val score:   0.527753992777633
Log Test score:  0.5309745055992375


In [18]:
log

LogisticRegressionCV(cv=3)

In [19]:
for pair in sorted(zip(X.columns,log.coef_[0]),key=lambda p: np.abs(p[1]),reverse=True):
    print("%30s | % .8f" % pair)

              previous_skipped |  0.06306652
     previous_session_position |  0.06172265
              session_position |  0.03304124
             previous_duration |  0.00099223
       previous_dyn_range_mean |  0.00097312
    previous_acoustic_vector_3 |  0.00096513
    previous_acoustic_vector_7 |  0.00096280
    previous_acoustic_vector_1 |  0.00096227
    previous_acoustic_vector_2 |  0.00095908
             previous_loudness |  0.00095761
                previous_tempo |  0.00093186
           previous_bounciness |  0.00093014
         previous_danceability |  0.00092788
        previous_beat_strength |  0.00092751
          previous_speechiness |  0.00092600
               previous_energy |  0.00092569
            previous_mechanism |  0.00092477
       previous_time_signature |  0.00092224
             previous_liveness |  0.00092197
             previous_flatness |  0.00092147
                  previous_key |  0.00091799
              previous_valence |  0.00091768
         p

In [20]:
# Train a Random Forest model and score
rfc = RandomForestClassifier(
    n_estimators=100
).fit(
    X_train,
    y_train
)

print("RFC Train score: %s" % rfc.score(X_train,y_train))
print("RFC Val score:   %s" % rfc.score(X_val,y_val))
print("RFC Test score:  %s" % rfc.score(X_test,y_test))

RFC Train score: 0.9653304542873896
RFC Val score:   0.7077175086556718
RFC Test score:  0.7068143912318322


In [21]:
for pair in sorted(zip(X.columns,rfc.feature_importances_),key=lambda p:p[1],reverse=True):
    print("%30s | % .8f" % pair)

              previous_skipped |  0.20488743
              session_position |  0.04416838
     previous_session_position |  0.04411371
        us_popularity_estimate |  0.01676524
                      liveness |  0.01497781
                      duration |  0.01494571
                         tempo |  0.01474448
             acoustic_vector_6 |  0.01458250
                      loudness |  0.01440538
                      flatness |  0.01438803
                  acousticness |  0.01437372
                   speechiness |  0.01436688
                       valence |  0.01429336
             acoustic_vector_1 |  0.01426562
             acoustic_vector_7 |  0.01416818
             previous_duration |  0.01415991
             acoustic_vector_5 |  0.01405948
                        energy |  0.01397709
             acoustic_vector_2 |  0.01395822
                     mechanism |  0.01385318
previous_us_popularity_estimate |  0.01384843
             acoustic_vector_4 |  0.01382768
         